In [ ]:
%%HTML
<!-- Improve projector display -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; }
.container {width:95% !important;}
</style>

# SoundLapse

In [ ]:
# -*- coding: utf-8 -*-
''' TimeLapse 2019 '''
import funcionestimelapse as tl
import file
import numpy as np
import os
import subprocess
import sys
import soundfile as audio
from scipy.io import wavfile
print ('Starting Sound-Lapse.')

## Soundscape input 
#### Select audios that will make up the audio capsule

In [ ]:
filez = file.filebrowser()

## Inputs
#### Input of the desired parameters for the creation of the representative sound capsule.

In [ ]:
in_t_inic      = float(input('Starting point:[min] ') )  # min. Instant timelapse begins.
in_t_segm      = float(input('Chunk duration:[seg] ') )  # sec. Duration of each chunk.
in_t_delta     = float(input('Chunk recording period :[min] '))  # min. Delta of time between chunks.
in_t_fade      = float(input('Crossfade duration:[seg] '))  # seg. Duration of each fades and crossfaders.
crossfade_type = int(input('Crossfade window shape:(1 Linear,2 Exponencial,3 Logarithmic) '))  # Crossfade Type.
cross_out      = tl.get_cross_out(crossfade_type)  # Obtaining crossfade type data.

## SoundLapse creation

In [ ]:
cont          = 0  # Starting bar progress.
for filepath in filez:
    in_sig                                  = audio.SoundFile(filepath, 'r')  # Audio reading.
    samp_freq                               = in_sig.samplerate  # Sample Rate.
    len_in_sig                              = len(in_sig)  # Signal length.
    # Preprocessed: Transformation of sec / min to samples.
    m_desc, m_fades, m_inic_desc, m_segment = tl.time_to_samples(in_t_inic, in_t_segm, in_t_delta, in_t_fade, samp_freq)
    stereo                                  = tl.check_stereo(in_sig)
    if stereo==True:
        if filepath == filez[0]:
            chunk_fadeout = np.zeros(shape=(1, 2))
            time_lapse_vector = np.zeros(shape=(1, 2))  # Pre-location of zeros vector for timelapse.
    else:
        if filepath == filez[0]:
            chunk_fadeout = np.zeros(shape=(1, ))
            time_lapse_vector = np.zeros(shape=(1, ))  # Pre-location of zeros vector for timelapse.
    fadein_vect, fadeout_vect, cross_out    = tl.get_crossfaders(crossfade_type, m_fades,stereo)  # Crossfaders vectors.
    # Segmentation
    n_chunks                                = tl.get_nchunks(len_in_sig, m_inic_desc, m_segment, m_desc)  # Number of timelapse chunks
    m_rest                                  = len_in_sig  # Muestras restantes del vector de audio original.
    time_lapse_vector, chunk_fadeout, cont  = tl.get_timelapse_one_file(in_sig, n_chunks, fadein_vect, fadeout_vect,
                                                                       m_inic_desc, m_segment, m_desc, m_fades,
                                                                       time_lapse_vector, filez, filepath,
                                                                       chunk_fadeout, cont)  # Generate TimeLapse

print ("Audio Segmentation Finished.")

## Export 
#### File is saved in audio format

In [ ]:
if stereo==True:
    outfile = 'Outputs/SoundLapse_'+str(int(in_t_inic))+'_'+str(int(in_t_segm))+'_'+str(int(in_t_delta))+'_'+str(int(in_t_fade))+'_'+'cross_' + cross_out
    f       = audio.SoundFile(outfile+'.wav', 'w',samplerate=samp_freq,channels=2,subtype='PCM_16',endian='BIG',format='WAV')
    f.write(time_lapse_vector)
    f.close()
else:
    outfile = 'Outputs/SoundLapse_'+str(int(in_t_inic))+'_'+str(int(in_t_segm))+'_'+str(int(in_t_delta))+'_'+str(int(in_t_fade))+'_'+'cross_' + cross_out
    f       = audio.SoundFile(outfile+'.wav', 'w',samplerate=samp_freq,channels=1,subtype='PCM_16',endian='BIG',format='WAV')
    f.write(time_lapse_vector)
    f.close()
print ("Exporting file " + outfile + ".wav")

### Stereo to mono
#### For the graphic representation of the Soundlapse

In [ ]:
if stereo ==True:
    sample_rate, samples = wavfile.read(outfile+'.wav')
    mono_samples         = tl.stereoToMono(samples)
    f_mono               = audio.SoundFile(outfile+"_mono.wav", 'w',samplerate=samp_freq,channels=1,subtype='PCM_16',endian='BIG',format='WAV')
    f_mono.write(mono_samples)
    f_mono.close()
print ('TimeLapse finished.')